In [ ]:

X.shape

In [ ]:
import torch
import hydra
from lib.utils.config import load_config
from lib.model.flame import FLAME
from lib.data.datamodule import DPHMDataModule
from lib.data.scheduler import CoarseToFineScheduler, FinetuneScheduler
from lib.model.flame import FLAME
from lib.model.logger import FlameLogger
from lib.model.loss import calculate_point2plane

cfg = load_config(
    "optimize",
    overrides=[
        "model.init_mode=kinect",
        "model.vertices_mask=full",
        "model.optimize_frames=10",
        "data=kinect",
        "data.scale=8",
        "data.batch_size=10",
        "data.start_frame_idx=19",
    ],
)
device = "cuda" if torch.cuda.is_available() else "cpu"
datamodule: DPHMDataModule = hydra.utils.instantiate(cfg.data, devie=device)
datamodule.setup()  # this now contains the intrinsics, camera and rasterizer
model: FLAME = hydra.utils.instantiate(cfg.model) 
model.init_renderer(camera=datamodule.camera, rasterizer=datamodule.rasterizer)
model = model.to(device)
fts: FinetuneScheduler = hydra.utils.instantiate(cfg.scheduler.finetune)
c2fs: CoarseToFineScheduler = hydra.utils.instantiate(cfg.scheduler.coarse2fine)

In [ ]:
# fetch single batch
iter_step = 0
c2fs.schedule_dataset(datamodule=datamodule, iter_step=iter_step)
fts.param_groups(model, iter_step=iter_step)
dataloader = datamodule.train_dataloader()
batch = next(iter(dataloader))

In [ ]:
fts.state.keys()

In [ ]:
import inspect
signature = inspect.signature(model.forward)
param_names = [param.name for param in signature.parameters.values()]
param_names

In [ ]:
import torch

# n = 100
A = torch.rand((700, 700))
B = torch.rand(700)
X = torch.linalg.solve(A, B)
x = torch.zeros((700), requires_grad=True)
# def foo(x):
#     return (A @ x - B)
# J = torch.autograd.functional.jacobian(foo, x)
# J

In [ ]:
x.grad

In [ ]:
_B = B.unsqueeze(-1)
(1/_B).T @ (A @ _B).T

In [ ]:
A

In [ ]:
from tqdm import tqdm
max_steps = 100000
x = torch.zeros((n), requires_grad=True)
# optimizer = torch.optim.Adam([x], lr=1.0) 
optimizer = torch.optim.LBFGS([x) 

for step in tqdm(range(max_steps)):
    optimizer.zero_grad()
    F = A @ x - B
    loss = torch.pow(F, 2).sum()
    # print(f"{step}) {loss}")
    loss.backward()
    optimizer.step()
print(f"{step}) {loss}")

In [ ]:
A.inverse() @ B

In [ ]:
(A.T @ A).inverse() @ (A.T @ B)

In [ ]:
torch.linalg.solve(A, B)

In [ ]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)
Q = 3*a**3 - b**2
external_grad = torch.tensor([1., 1.])
Q.sum().backward()

In [ ]:
a = torch.randn(5, requires_grad=True)
b = 2 * a
c = b ** 2  # replace this with c = b + 2 and the autograd error will go away
print(b._version)
b = b + 1
print(b._version)
b += 1  # inplace operation!
print(b._version)
# c.sum().backward()